In [91]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [111]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [112]:
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config
import pprint
from IPython.display import IFrame
from IPython.display import display

In [90]:
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from fuzzywuzzy import process

/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [82]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!") 

In [115]:
model = load("spotify_17.pickle")

In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [5]:
url = "https://www.billboard.com/charts/hot-100/"
response = requests.get(url)
soup = BeautifulSoup(response.content)

In [6]:
def extract_billboard_100(soup):
    
    titles = []
    artists = []

    for title in soup.select('h3.c-title.a-no-trucate.a-font-primary-bold-s'):
        titles.append(title.text.strip())

    for artist in soup.select('span.c-label.a-no-trucate.a-font-primary-s'):
        artists.append(artist.text.strip())

    billboard_df = pd.DataFrame({
        'Title': titles,
        'Artist': artists
    })

    return billboard_df

In [7]:
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [8]:
frank_ocean_mix = get_playlist_tracks("spotify", "37i9dQZF1EIYHlfEDwnhiL")
best_indie_2016 = get_playlist_tracks("spotify", "37i9dQZF1DX0XnW5YpwS0o")
jordan_critical_mass = get_playlist_tracks("Jordan Chiang", "77sN6AyfT0rPWY2vtekfq4")
temper_trap_radio = get_playlist_tracks("spotify", "37i9dQZF1E4nafnsLnjXjb")
hip_hop_classics = get_playlist_tracks("Wolfgang Wee", "696fgrOcxfuSWtnLW5VOYw")
oldies_throwbacks = get_playlist_tracks("Indiemono", "1TNg7JCxifAjwrnQARimex")
cumbia_psicodelica_peruana = get_playlist_tracks("xagosaez", "1B59sc3clzJU65zovuIwdY")
tulum_vibes = get_playlist_tracks("spotify", "37i9dQZF1DX9pP7cC2liKw")
samurai_champloo = get_playlist_tracks("Cold Soda Music", "6oLxvkZALONjDIDXXk8ysd")
deep_tech_house = get_playlist_tracks("Ruben Moritz", "6SQVD0iCvTiUons1e2xf1J")
big_indie_list = get_playlist_tracks("Marc Krolla", "1taIvhOEgs2Rr2sOHyvpGc")
biggest_hits = get_playlist_tracks("groosteruk", "6FTC0TcQ0WCrdHQMR5Lubo")
best_world = get_playlist_tracks("Cumbancha", "4RtQxCJm13QnwLoF4HEoFR")

In [9]:
all_lists = frank_ocean_mix.copy()
all_lists.extend(best_indie_2016)
all_lists.extend(jordan_critical_mass)
all_lists.extend(temper_trap_radio)
all_lists.extend(hip_hop_classics)
all_lists.extend(oldies_throwbacks)
all_lists.extend(cumbia_psicodelica_peruana)
all_lists.extend(tulum_vibes)
all_lists.extend(samurai_champloo)
all_lists.extend(deep_tech_house)
all_lists.extend(big_indie_list)
all_lists.extend(biggest_hits)
all_lists.extend(best_world)

In [114]:
pl_ids = []
for i in all_lists:
    pl_ids.append(i['track']['id'])
    
chunk_size = 100
pl_ids_chopped = [pl_ids[i:i+chunk_size] for i in range(0,len(pl_ids), 100)]

af_list = []
for i in pl_ids_chopped:
    af_list.extend(sp.audio_features(i))
af_df = pd.DataFrame(af_list)
af_df.to_csv('af_df.csv', index=False)

In [29]:
X = pd.DataFrame(af_list).select_dtypes(include=['number'])
X.drop('duration_ms',axis=1,inplace=True)
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)

In [30]:
kmeans = KMeans(n_clusters=18, random_state=1234)
kmeans.fit(X_scaled_df)
labels = kmeans.labels_

/Users/jordanchiang/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [31]:
clusters = kmeans.predict(X_scaled_df)
pd.Series(clusters).value_counts().sort_index()
X["cluster"] = kmeans.labels_

In [78]:
def play_user_song():
    user_input = input('Please enter a song name (include artist name for more accuracy): \n').strip().lower().replace(',',' ')
    
    song_search = sp.search(user_input, type="track", limit=1, market="GB")
    track_id = song_search["tracks"]["items"][0]["id"]
    
    print(f"You've chosen: {song_search['tracks']['items'][0]['name']}, by {song_search['tracks']['items'][0]['artists'][0]['name']}.")
    
    iframe = IFrame(src="https://open.spotify.com/embed/track/" + track_id,
                    width="320",
                    height="80",
                    frameborder="0",
                    allowtransparency="true",
                    allow="encrypted-media")
    display(iframe)
    
    return song_search

In [74]:
def play_next_song(song_search):
    song_artist = song_search["tracks"]["items"][0]["artists"][0]["name"].lower()
    song_title = song_search["tracks"]["items"][0]["name"].lower()
    bb100_df = extract_billboard_100(soup)
  
    # check if song artist and title is from billboard top 100 
    bb100_match = bb100_df[(bb100_df['Title'].str.lower() == song_title) & (bb100_df['Artist'].str.lower() == song_artist)]
    if not bb100_match.empty:
        
        # if yes, play next random song from bb100
        random_bb100 = bb100_df.sample()
        next_bb100_title = random_bb100['Title']
        next_bb100_artist = random_bb100['Artist'].str.slice()
        bb100_next_track = sp.search(q="{next_bb100_title}, {next_bb100_artist}", type="track", limit=1, market="GB")
        bb100_next_id = bb100_next_track['tracks']['items'][0]['id']
        print("Your song is hot!\n")
        print(f"We can also recommend {next_bb100_title.values[0]}, by {next_bb100_artist.values[0]} from the Billboard Hot 100 list.\n")

        iframe_bb100 = IFrame(src="https://open.spotify.com/embed/track/"+bb100_next_id,
               width="320", 
               height="80",
               frameborder="0",
               allowtransparency="true",
               allow="encrypted-media")
        display(iframe_bb100)
        
  
    # else pick next random song from big playlist to play
    else:
        # get and scale audio features of user input song to predict cluster
        track_id = song_search["tracks"]["items"][0]["id"]
        input_af = pd.DataFrame(
            sp.audio_features(track_id)).select_dtypes(include=['number']).drop('duration_ms',axis=1)
        af_scaled = pd.DataFrame(scaler.transform(input_af), columns = input_af.columns)
        af_scaled["cluster"] = kmeans.predict(af_scaled)
        
        # get a inner merged df with just the matching cluster of songs and their ids
        song_cluster_df = pd.DataFrame(X)
        song_cluster_df = song_cluster_df[song_cluster_df['cluster'] == af_scaled["cluster"].iloc[0]]
        next_song_df = pd.DataFrame(af_list).merge(
            song_cluster_df['cluster'], left_index=True, right_index=True, how='inner')
        
        # choose random song from clustered df and play
        next_song = next_song_df.sample()
        next_song_id = str(next_song['id'].values[0])
        next_song_details = sp.track(next_song_id)
        next_song_artist = next_song_details['artists'][0]['name']
        next_song_title = next_song_details['name']
        
        print("You've got great taste!\n")
        print(f"We can also recommend {next_song_title}, by {next_song_artist} from a similar genre.\n")
        
        iframe_playlist = IFrame(src="https://open.spotify.com/embed/track/"+next_song_id,
               width="320", 
               height="80",
               frameborder="0",
               allowtransparency="true",
               allow="encrypted-media")
        display(iframe_playlist)

In [81]:
song_search = play_user_song()

Please enter a song name (include artist name for more accuracy): 
balling, trevor
You've chosen: You, by Trevor Ballinger.


In [80]:
next_song = play_next_song(song_search)

You've got great taste!

We can also recommend Back To Us, by Two Another from a similar genre.

